In [1]:
load_model = 'regnet_y_3_2gf_based_model2.pt'

data_dir = '../../../stanford_dogs_new/'

In [2]:
# import numpy as np

from IPython.display import Image

import os
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision.models as models
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

from torchvision.datasets import ImageFolder
from PIL import Image

from torch.autograd import Variable

from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy


from IPython.display import Image
from IPython.display import Markdown

from PIL import Image
import PIL.Image

import pandas as pd

# sklearn metrics
from sklearn import metrics
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import f1_score


## load the regnet_y_3_2gf based model

In [3]:
regnet_y_3_2gf_loaded = models.regnet_y_3_2gf(pretrained=True)

for param in regnet_y_3_2gf_loaded.parameters():
    param.requires_grad = False

regnet_y_3_2gf_loaded.fc = nn.Linear(in_features=1512, out_features=120, bias=True)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
regnet_y_3_2gf_loaded = regnet_y_3_2gf_loaded.to(device)



regnet_y_3_2gf_loaded.load_state_dict(torch.load('regnet_y_3_2gf_based_model2.pt'), strict=False)
regnet_y_3_2gf_loaded.eval()

Downloading: "https://download.pytorch.org/models/regnet_y_3_2gf-b5a9779c.pth" to /home/rka73/.cache/torch/hub/checkpoints/regnet_y_3_2gf-b5a9779c.pth


RegNet(
  (stem): SimpleStemIN(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (trunk_output): Sequential(
    (block1): AnyStage(
      (block1-0): ResBottleneckBlock(
        (proj): ConvNormActivation(
          (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(2, 2), bias=False)
          (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (f): BottleneckTransform(
          (a): ConvNormActivation(
            (0): Conv2d(32, 72, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): BatchNorm2d(72, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU(inplace=True)
          )
          (b): ConvNormActivation(
            (0): Conv2d(72, 72, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=3, bias=False)
            (1): B

In [4]:
model = regnet_y_3_2gf_loaded

In [5]:
# preprocess the image for the model
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
    )
])

In [6]:
path = '../../../stanford_dogs_new/test'

list_subfolders_with_paths = [f.path for f in os.scandir(path) if f.is_dir()]

# gets the breed name from subfolders name like 'n02099429-curly-coated_retriever'
def rename(name):
    return ' '.join(' '.join(name.split('-')[1:]).split('_'))

per_class_accuracy = dict()
total_accuracy = 0

y_true = []
y_pred = []

for breed_dir in list_subfolders_with_paths:
    
    breed = rename(breed_dir.split('/')[-1])
    
    images = os.listdir(breed_dir)
    
    matched = False
    matches = 0
    total_images = len(images)

    for image in images:

        input_image = Image.open(breed_dir + '/' + image)


        input_tensor = preprocess(input_image)
        if torch.cuda.is_available():
            input_tensor = Variable(input_tensor.cuda())

        input_batch = input_tensor.unsqueeze(0)
        out = model(input_batch)

        probabilities = torch.nn.functional.softmax(out[0], dim=0)
        # print(probabilities)

        with open("../../stanford_dogs_breeds_classes_final.txt", "r") as f:
            categories = [s.strip() for s in f.readlines()]

        predicted_breeds = []
        top1_prob, top1_catid = torch.topk(probabilities, 1)
        for i in range(top1_prob.size(0)):
            # predicted_breeds.append([categories[top3_catid[i]], top3_prob[i].item()*100])
            predicted_breed = categories[top1_catid[i]]
            y_true.append(breed)
            y_pred.append(predicted_breed)

        # list to be used directly by the application (predicted_breed, probability)
        # print("predicted_breeds are: \n", predicted_breed)
        if breed == predicted_breed:
            matched = True
            matches = matches + 1

        per_class_accuracy[breed] = matches / total_images * 100

In [7]:
df = pd.DataFrame(list(zip(y_true, y_pred)),
               columns =['Actual_breed', 'Predicted_breed'])

df

,Actual_breed,Predicted_breed
0,Pomeranian,Pomeranian
1,Pomeranian,Pomeranian
2,Pomeranian,Pomeranian
3,Pomeranian,Pomeranian
4,Pomeranian,Pomeranian
...,...,...
2105,papillon,papillon
2106,papillon,papillon
2107,papillon,papillon
2108,papillon,papillon


## Confusion Matrix

In [8]:
# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


[[22  0  0 ...  0  0  0]
 [ 0 17  0 ...  0  0  0]
 [ 0  0 21 ...  0  0  0]
 ...
 [ 0  0  0 ... 16  0  0]
 [ 0  0  0 ...  0 13  0]
 [ 0  0  0 ...  0  0 13]]


## Precision, Recall and F1

In [9]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

                                precision    recall  f1-score   support

                  Afghan hound      1.000     0.917     0.957        24
           African hunting dog      0.944     1.000     0.971        17
                      Airedale      0.778     1.000     0.875        21
American Staffordshire terrier      0.857     0.706     0.774        17
                   Appenzeller      1.000     0.438     0.609        16
            Australian terrier      0.789     0.750     0.769        20
            Bedlington terrier      0.950     1.000     0.974        19
          Bernese mountain dog      0.778     0.955     0.857        22
              Blenheim spaniel      0.950     1.000     0.974        19
                 Border collie      0.625     0.667     0.645        15
                Border terrier      0.944     0.944     0.944        18
                   Boston bull      0.947     0.947     0.947        19
          Bouvier des Flandres      0.786     0.733     0.759  

## Macro F1 Score

In [10]:
from sklearn.metrics import f1_score

f1_score(y_true, y_pred, average='macro')


0.8446974192288966

## Micro F1 Score

In [11]:
f1_score(y_true, y_pred, average='micro')


0.852132701421801

## Weighted F1 Score

In [12]:
f1_score(y_true, y_pred, average='weighted')

0.8464295696927688

## Cohen Kappa Score

In [13]:
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(y_true, y_pred)

0.8508608936656658

## Total Accuracy calculated from per class accuracy

In [14]:
# calculated from per class accuracy
total_accuracy = sum(per_class_accuracy.values())/120
total_accuracy

84.8690484495438

## Per Breed Accuracy

### Top 10 Breeds with least accuracy

In [15]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1])[:k]

[('Eskimo dog', 6.666666666666667),
 ('collie', 37.5),
 ('Lakeland terrier', 40.0),
 ('Lhasa', 42.10526315789473),
 ('Appenzeller', 43.75),
 ('Walker hound', 50.0),
 ('toy terrier', 50.0),
 ('miniature poodle', 56.25),
 ('cairn', 60.0),
 ('giant schnauzer', 62.5)]

### Top 10 Breeds with most accuracy

In [16]:
k = 10
sorted(per_class_accuracy.items(), key=lambda x: x[1], reverse=True)[:k]

[('Japanese spaniel', 100.0),
 ('Brabancon griffon', 100.0),
 ('Blenheim spaniel', 100.0),
 ('Irish setter', 100.0),
 ('clumber', 100.0),
 ('schipperke', 100.0),
 ('Old English sheepdog', 100.0),
 ('Pembroke', 100.0),
 ('keeshond', 100.0),
 ('Samoyed', 100.0)]